<a href="https://colab.research.google.com/github/Kakaoko/Sandwich-Panel-Investigation/blob/master/Sandwich_Panel_Transfer_Matrix_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt

#Material property
#PLA for 3D Printing
global ela_pla, den_pla, c_pla, c_air, zc_air, theta
e_pla = 3.5 * 10**9
d_pla = 1240
c_pla = 2220
c_air = 344
zc_air = 409.4

#Thin Elastic Panel Transfer Matrix
class panel :
    def __init__(self, f, b, h, L) :
        self.f = f
        self.b = b
        self.h = h
        self.L = L

    def w(self) :
        return 2 * np.pi * self.f

    def k_pla(self) :
        return 2 * np.pi * self.f / c_pla

    def k_air(self) :
        return 2 * np.pi * self.f / c_air

    def B(self) :
        return e_pla * self.b * self.h**3 / (12 * self.L)

    def Zp(self) :
        return panel(self.f, self.b, self.h, self.L).B() * (panel(self.f, self.b, self.h, self.L).k_air()**4 * np.sin(theta)**4 - panel(self.f, self.b, self.h, self.L).k_pla()**4) / (panel(self.f, self.b, self.h, self.L).w() * 1j)

    def T_panel(self) :
        return np.array([[1, panel(self.f, self.b, self.h, self.L).Zp()], [0 ,1]])

#Fluid Layer Transfer Matrix
class fluid :
    def __init__(self, f, d) :
        self.f = f
        self.d = d

    def w(self) :
        return 2 * np.pi * self.f

    def k_air(self) :
        return 2 * np.pi * self.f / c_air

    def m00(self) :
        return np.cosh(fluid(self.f, self.d).k_air() * self.d * np.cos(theta) * 1j)

    def m01(self) :
        return zc_air * np.sinh(fluid(self.f, self.d).k_air() * self.d * np.cos(theta) * 1j) / np.cos(theta)

    def m10(self) :
        return (np.sin(fluid(self.f, self.d).k_air() * self.d * np.cos(theta) * 1j)) * np.cos(theta) / zc_air

    def m11(self) :
        return fluid(self.f, self.d).m00()

    def T_fluid(self) :
        return np.array([[fluid(self.f, self.d).m00(), fluid(self.f, self.d).m01()], [fluid(self.f, self.d).m10(), fluid(self.f, self.d).m11()]])

#Porous Layer Transfer Matrix
class porous :
    def __init__(self, f, d, sigma) :
        self.f = f
        self.d = d
        self.sigma = sigma

    def w(self) :
        return 2 * np.pi * self.f

    def k_air(self) :
        return 2 * np.pi * self.f / c_air

    def m00(self) :
        return np.cosh(porous(self.f, self.d, self.sigma).k_air() * self.d * np.cos(theta) * 1j)

    def m01(self) :
        return zc_air * np.sinh(porous(self.f, self.d, self.sigma).k_air() * self.d * np.cos(theta) * 1j) / (np.cos(theta) * self.sigma)

    def m10(self) :
        return (np.sin(porous(self.f, self.d, self.sigma).k_air() * self.d * np.cos(theta) * 1j)) * np.cos(theta) * self.sigma / zc_air

    def m11(self) :
        return porous(self.f, self.d, self.sigma).m00()

    def T_porous(self) :
        return np.array([[porous(self.f, self.d, self.sigma).m00(), porous(self.f, self.d, self.sigma).m01()], [porous(self.f, self.d, self.sigma).m10(), porous(self.f, self.d, self.sigma).m11()]])

#Panel -> Air -> Porous -> Air -> panel
#Graph Plotting
def para_input() :
    b = float(input("단면적의 가로 길이를 입력하세요(m) : "))
    h = float(input("단면적의 세로 길이를 입력하세요(m) : "))
    L = float(input("Panel의 두께를 입력하세요(m) : "))
    d = float(input("공기층의 두께를 입력하세요(m) : "))
    sigma = float(input("Porosity를 입력하세요(0 ~ 1) : "))
    theta = float(input("입사각을 입력하세요(rad) : "))
    return b, h, L, d, sigma, theta

b, h, L, d, sigma, theta = para_input()
tau = []
for f in range(50, 1600) :
    T = np.matmul(np.matmul(np.matmul(panel(f, b, h, L).T_panel(), fluid(f, d).T_fluid()), np.matmul(porous(f, d, sigma).T_porous(), fluid(f, d).T_fluid())), panel(f, b, h, L).T_panel())
    tau.append(4 * np.absolute(T[0][0] + T[0][1] / zc_air + zc_air * T[1][0] + T[1][1])**-2)

TL = []
for i in range(len(tau)) :
    TL.append(- 10 * np.log10(tau[i]))

plt.plot(np.arange(50, 1600), TL, ":")
plt.title("Sandwich Panel(Panel -> Air -> Porous -> Air -> Panel)")
plt.xlabel("Frequency(Hz)")
plt.ylabel("TL(dB)")
plt.show()